<h1> Assignment #1 </h1>

You are an engineer in the logistic company. You have a robot which is capable of cargo transportation. Now you use it to carry the equipment over the cluttered warehouse. 

Your task is to train bot to find the shortest path to the required location (lower right corner) so that it does not crash into obstacles.  

---

Submit `{name}_{surname}.py` script with `find_path(path_to_infile, path_to_outfile)` function.

**You have to solve it using MDP.**

### Input:

`{infile}.txt` file with the field description. Elements of the field are separated by space. For example: 
```
2 1 0 1 0
0 1 0 1 0
0 0 0 0 0
0 1 0 1 0
0 1 0 1 0
```

* `0` - blank space, we may move objects here
* `1` - obstackles, object can not be over that position
* `2` - actual object shape, does not change, moved as a solid object

As our goal is to move object to the most lower-right position, sequence of our steps will be the following:  
`D D R R R R D D`

Meaning:  
`D` - (move) Down;  
`U` - (move) Up;  
`R` - (move) Right;  
`L` - (move) Left.

### Output:

Sequence that leads to the right lower corner in the least number of steps. Written in `{outfile}.txt`. If there is no path, write `No path` in the file.

### Examples

#### First case

`input.txt`:

```
0 1 0 1 0
0 1 0 1 0
0 0 0 0 0
0 0 0 0 0
0 1 2 1 0
0 1 2 0 0
```

`out.txt`:

```
U U R R D D
```

#### Second case

`input.txt`:

```
2 2 1 0 1 0
0 2 1 0 1 0
0 2 0 0 0 0
0 0 0 0 0 0
0 0 1 0 1 0
0 0 1 0 0 0
```

`out.txt`:

```
No path
```

#### Third case

`input.txt`:

```
2 2 1 0 1 0
0 2 1 0 1 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 1 0 1 0
0 0 1 0 0 0
```

`out.txt`:

```
No path
```

#### Fourth case

`input.txt`:

```
2 2 1 0 1 0
0 2 0 0 0 0
0 2 0 0 0 0
0 0 0 0 0 0
0 0 0 0 1 0
0 0 1 0 0 0
```

`out.txt`:

```
D D R R U R R D D
```